# Модели для распознования пневмонии

Датасет организован в 3 папки (train, test, val) и содержит подпапки для каждой категории изображений (Пневмония/Нормальные). Есть 5866 рентгеновских изображений в формате JPEG и 2 категории (Пневмония/Нормальные).

Рентгеновские снимки клетки (переднего-заднего плана) были отобраны из ретроспективных когорт педиатрических пациентов в возрасте от одного до пяти лет из Женского и детского медицинского центра Гуанчжоу. Все рентгеновские снимки грудной клетки проводились в рамках обычной клинической помощи пациентам.

Для анализа рентгеновских изображений грудной клетки все ренгенограммы грудной клетки были первоначально проверены на контроль качества путем удаления всех низкокачественных или нечитаемых сканов. Зате диагнозы для изображений были оценены двумя врачами-экспертами, прежде чем они были очищены для дальнейшего исопльзования. Чтобы учесть любые ошибки оценки, набор оценок также был проверен третьим экспертом.

## Импортирование пакетов

In [1]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
import matplotlib.pyplot as plt
from math import ceil
import tensorflow as tf
import time
from PIL import Image
from tensorflow.keras.metrics import F1Score, Precision, Recall
import pandas as pd

2024-05-01 08:06:18.262155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Определение директорий для наборов данных

In [2]:
train_dir = 'chest_xray/train/'
val_dir = 'chest_xray/val/'
test_dir = 'chest_xray/test/'

## Общие параметры

In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 15
nb_train_samples = 5032  
nb_validation_samples = 200

# Создание генератора данных для обучения с аугментацией
train_datagen = ImageDataGenerator(
    rescale=1/255,  # Нормализация значений пикселей изображений [0, 255] -> [0, 1]
    horizontal_flip=True,  # Случайное горизонтальное отражение изображений для аугментации
    brightness_range=[0.5, 2.0],  # Диапазон для случайного изменения яркости изображений
    width_shift_range=0.2,  # Диапазон для случайного горизонтального сдвига изображений
    rotation_range=20,  # Диапазон для случайного поворота изображений
    zoom_range=0.2,  # Диапазон для случайного масштабирования изображений
    shear_range=0.1,  # Диапазон для случайного сдвига изображений
    fill_mode='nearest'  # Метод заполнения новых пикселей, которые могут появиться после аугментации
)

# Создание генератора данных для валидации и тестирования без аугментации, только нормализация
val_datagen = ImageDataGenerator(rescale=1./255)

# Подготовка генератора данных для обучения
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,  # Целевой размер изображений, к которому они будут изменены
    batch_size=BATCH_SIZE,  # Размер батча
    class_mode='binary'  # Режим классификации ('binary' для бинарной классификации)
)

# Подготовка генератора данных для валидации
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,  # Целевой размер изображений
    batch_size=BATCH_SIZE,  # Размер батча
    class_mode='binary'  # Режим классификации
)

# Подготовка генератора данных для тестирования
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,  # Целевой размер изображений
    batch_size=BATCH_SIZE,  # Размер батча
    class_mode='binary'  # Режим классификации
)

Found 5032 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## CNN

### Настройка модели CNN

In [4]:
cnn_model = Sequential()
# Создаем стек слоев, каждый из которых имеет один входной и один выходной вектор (последовательная модель)

cnn_model.add(Conv2D(32, (3, 3), input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
#Задаем слой 2D свертки. 
#Первый аргумент - представляет количество фильтров, на основе которых обучается сверточный уровень, так называемая глубина тензора
#Второй аргумент - размер ядра фильтра. В данном случае он представляет собой квадратную матрицу 3*3
#Третий аргумент - размер входных данных - длина * ширина * 3 (150 * 150 * 3)

cnn_model.add(Activation('relu'))
# Слой функции активации (в нашем случае - функции активации RELU)
# Заметим, что ничего не мешает вписать эту функцию внутрь предыдущего слоя, результат от этого не изменится

cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
# Операция выбора максимального значения из соседних, "агрессивное" уменьшение разрешения карты признаков
# В данном случае выбор максимального среди четырех значений квадратной матрицы 2*2

cnn_model.add(Conv2D(32, (3, 3)))
#Задаем слой 2D свертки. 
#Первый аргумент - представляет количество фильтров, на основе которых обучается сверточный уровень, так называемая глубина тензора
#Второй аргумент - размер ядра фильтра. В данном случае он представляет собой квадратную матрицу 3*3

cnn_model.add(Activation('relu'))
# Слой функции активации (в нашем случае - функции активации RELU)
# Заметим, что ничего не мешает вписать эту функцию внутрь предыдущего слоя, результат от этого не изменится

cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
# Операция выбора максимального значения из соседних, "агрессивное" уменьшение разрешения карты признаков
# В данном случае выбор максимального среди четырех значений квадратной матрицы 2*2

cnn_model.add(Conv2D(64, (3, 3)))
#Задаем слой 2D свертки. 
#Первый аргумент - представляет количество фильтров, на основе которых обучается сверточный уровень, так называемая глубина тензора
#Второй аргумент - размер ядра фильтра. В данном случае он представляет собой квадратную матрицу 3*3

cnn_model.add(Activation('relu'))
# Слой функции активации (в нашем случае - функции активации RELU)
# Заметим, что ничего не мешает вписать эту функцию внутрь предыдущего слоя, результат от этого не изменится

cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
# Операция выбора максимального значения из соседних, "агрессивное" уменьшение разрешения карты признаков
# В данном случае выбор максимального среди четырех значений квадратной матрицы 2*2

cnn_model.add(Flatten())
# Преобразует входные данные в один большой одномерный массив

cnn_model.add(Dense(64))
# Обычный слой нейронной сети с плотным соединением
# Первый аргумент -  Положительное число. Размерность выходного пространства. Он определяет, сколько нейронов будет в этом слое

cnn_model.add(Activation('relu'))
# Слой функции активации (в нашем случае - функции активации RELU)
# Заметим, что ничего не мешает вписать эту функцию внутрь предыдущего слоя, результат от этого не изменится

cnn_model.add(Dropout(0.5))
# Данный слой зануляет случайно выбранные признаки
# Первый аргумент соответствует доле отбрасываемых признаков

cnn_model.add(Dense(1))
# Обычный слой нейронной сети с плотным соединением
# Первый аргумент -  Положительное число. Размерность выходного пространства. Он определяет, сколько нейронов будет в этом слое
cnn_model.add(Activation('sigmoid'))
# Слой функции активации (в нашем случае - функции активации sigmoid)
# Заметим, что ничего не мешает вписать эту функцию внутрь предыдущего слоя, результат от этого не изменится

cnn_model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 222, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 109, 109, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 52, 52, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,768,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,797,665 (10.67 MB)

 Trainable params: 2,797,665 (10.67 MB)

 Non-trainable params: 0 (0.00 B)

## ResNet50

### Настройка модели ResNet50

In [5]:
def get_resnet_model():
    # Загружаем базовую модель ResNet50V2 с предобученными весами 'imagenet', без верхнего уровня (include_top=False)
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    
    # Замораживаем все слои базовой модели, чтобы их веса не изменялись во время обучения
    for layer in base_model.layers:
        layer.trainable = False
    
    # Добавляем глобальный средний слой пуллинга после базовой модели
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Добавляем полносвязный слой с 512 нейронами и функцией активации ReLU
    x = Dense(512, activation='relu')(x)
    
    # Добавляем слой Dropout с вероятностью 0.5 для регуляризации
    x = Dropout(0.5)(x)
    
    # Добавляем выходной слой с одним нейроном и сигмоидной активацией для бинарной классификации
    predictions = Dense(1, activation='sigmoid')(x)
    
    # Создаем финальную модель, объединяя базовую модель и новые добавленные слои
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

## Обучение и сравнение моделей CNN и ResNet50

In [6]:
# Компиляция модели CNN
cnn_model.compile(
    optimizer='adam',  # Оптимизатор Adam для обучения модели
    loss='binary_crossentropy',  # Функция потерь для задачи бинарной классификации
    metrics=[
        'accuracy',  # Метрика для отслеживания точности
        Precision(name='precision'),  # Метрика для отслеживания точности (precision)
        Recall(name='recall')  # Метрика для отслеживания полноты (recall)
    ]
)

# Получаем модель ResNet50V2
resnet_model = get_resnet_model()

# Компиляция модели ResNet50V2
resnet_model.compile(
    optimizer='adam',  # Оптимизатор Adam для обучения модели
    loss='binary_crossentropy',  # Функция потерь для задачи бинарной классификации
    metrics=[
        'accuracy',  # Метрика для отслеживания точности
        Precision(name='precision'),  # Метрика для отслеживания точности (precision)
        Recall(name='recall')  # Метрика для отслеживания полноты (recall)
    ]
)


In [9]:
def train_model_with_timing(model, train_generator, val_generator, test_generator, epochs, model_name):
    start_time = time.time()  # Запоминаем время начала обучения
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[ModelCheckpoint(f"{model_name}_best_model.keras", monitor='val_accuracy', save_best_only=True)]  # Сохранение наилучшей модели
    )
    end_time = time.time()  # Запоминаем время окончания обучения
    training_time = end_time - start_time  # Вычисляем время обучения

    # Вычисляем F1 для валидации
    val_precision = history.history.get('val_precision', [-1])[-1]  # Точность на валидационных данных
    val_recall = history.history.get('val_recall', [-1])[-1]  # Полнота на валидационных данных
    val_f1_score = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) != 0 else 0  # Вычисление F1-метрики

    # Вычисляем F1 для теста
    test_results = model.evaluate(test_generator)  # Оценка модели на тестовых данных
    test_precision = test_results[1]  # Точность на тестовых данных
    test_recall = test_results[2]  # Полнота на тестовых данных
    test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0  # Вычисление F1-метрики

    print(f"Время обучения модели {model_name}: {training_time:.2f} секунд.")  # Вывод времени обучения
    return {
        "train_accuracy": history.history['accuracy'][-1],  # Точность на обучающих данных
        "train_f1": val_f1_score,  # F1-метрика на валидационных данных
        "test_accuracy": test_results[0],  # Точность на тестовых данных
        "test_f1": test_f1_score,  # F1-метрика на тестовых данных
        "training_time": f"{training_time:.2f} s"  # Время обучения в секундах
    }

# Обучение и оценка моделей
cnn_metrics = train_model_with_timing(cnn_model, train_generator, val_generator, test_generator, EPOCHS, "CNN")
resnet_metrics = train_model_with_timing(resnet_model, train_generator, val_generator, test_generator, EPOCHS, "ResNet50")

# Создание и вывод таблицы с результатами
results_df = pd.DataFrame({
    'Метрика/Модель': ['Train accuracy', 'Train F1 average', 'Test accuracy', 'Test F1 average', 'Training time (s)'],
    'CNN': [
        cnn_metrics['train_accuracy'],  # Точность на обучающих данных для CNN
        cnn_metrics['train_f1'],  # F1-метрика на валидационных данных для CNN
        cnn_metrics['test_accuracy'],  # Точность на тестовых данных для CNN
        cnn_metrics['test_f1'],  # F1-метрика на тестовых данных для CNN
        cnn_metrics['training_time']  # Время обучения для CNN
    ],
    'ResNet50': [
        resnet_metrics['train_accuracy'],  # Точность на обучающих данных для ResNet50
        resnet_metrics['train_f1'],  # F1-метрика на валидационных данных для ResNet50
        resnet_metrics['test_accuracy'],  # Точность на тестовых данных для ResNet50
        resnet_metrics['test_f1'],  # F1-метрика на тестовых данных для ResNet50
        resnet_metrics['training_time']  # Время обучения для ResNet50
    ]
})

print(results_df)  # Вывод таблицы с результатами

Epoch 1/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 265s 835ms/step - accuracy: 0.9018 - loss: 0.2597 - precision: 0.9469 - recall: 0.9199 - val_accuracy: 0.8650 - val_loss: 0.3261 - val_precision: 0.8230 - val_recall: 0.9300
Epoch 2/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 245s 763ms/step - accuracy: 0.9029 - loss: 0.2287 - precision: 0.9529 - recall: 0.9148 - val_accuracy: 0.9050 - val_loss: 0.2985 - val_precision: 0.8584 - val_recall: 0.9700
Epoch 3/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 235s 737ms/step - accuracy: 0.8942 - loss: 0.2531 - precision: 0.9487 - recall: 0.9085 - val_accuracy: 0.9150 - val_loss: 0.2638 - val_precision: 0.9192 - val_recall: 0.9100
Epoch 4/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 235s 736ms/step - accuracy: 0.9049 - loss: 0.2460 - precision: 0.9506 - recall: 0.9199 - val_accuracy: 0.8750 - val_loss: 0.3082 - val_precision: 0.8205 - val_recall: 0.9600
Epoch 5/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 234s 734ms/step - accuracy: 0.9124 - loss: 0.2243 - precision: 0.9554 - recall: 0.9283 - val_accuracy: 0

In [11]:
# Сохранение модели ResNet-50
resnet_model.save_weights('ResNet50_best_model.keras')